<a href="https://colab.research.google.com/github/champtanapat/COS3109/blob/main/week_7_Naive_bayes_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to build and apply Naive Bayes classificator for spam filtering 


In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
ROOT = "/content/Gdrive"
drive.mount(ROOT)



Mounted at /content/Gdrive


Dataset from https://archive.ics.uci.edu/ml/datasets/sms+spam+collection

In [ ]:
sms_data = pd.read_csv('/content/Gdrive/MyDrive/Colab Notebooks/SMSSpamCollection', header=None, sep='\t', names=['Label', 'SMS'])


In [ ]:
sms_data.head()

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
sms_data.shape

(5572, 2)

In [ ]:
sms_data.groupby('Label').count()

,SMS
Label,
ham,4825
spam,747


# Prepare data

In [ ]:
sms_data_clean = sms_data.copy()

In [ ]:
sms_data_clean['SMS'] = sms_data_clean['SMS'].str.replace('\W+', ' ').str.replace('\s+', ' ').str.strip()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [ ]:
sms_data_clean['SMS'] = sms_data_clean['SMS'].str.lower()
print(sms_data_clean['SMS'])

In [ ]:
sms_data_clean['SMS'] = sms_data_clean['SMS'].str.split()
#print(sms_data_clean['SMS'])

In [ ]:
sms_data_clean['SMS'].head()

0    Go until jurong point crazy Available only in ...
1                              Ok lar Joking wif u oni
2    Free entry in 2 a wkly comp to win FA Cup fina...
3          U dun say so early hor U c already then say
4    Nah I don t think he goes to usf he lives arou...
Name: SMS, dtype: object

In [ ]:
sms_data_clean['Label'].value_counts() / sms_data.shape[0] * 100


ham     86.593683
spam    13.406317
Name: Label, dtype: float64

# Split to train and test data

In [108]:
train_data = sms_data_clean.sample(frac=0.8,random_state=1).reset_index(drop=True)
test_data = sms_data_clean.drop(train_data.index).reset_index(drop=True)
train_data = train_data.reset_index(drop=True)


In [109]:
train_data['Label'].value_counts() / train_data.shape[0] * 100

ham     86.54105
spam    13.45895
Name: Label, dtype: float64

In [110]:
train_data.shape

(4458, 2)

In [111]:
test_data['Label'].value_counts() / test_data.shape[0] * 100

ham     86.983842
spam    13.016158
Name: Label, dtype: float64

In [112]:
test_data.shape

(1114, 2)

In [146]:
test_data.head()

     Label  SMS
0      ham  NaN
1      ham  NaN
2     spam  NaN
3      ham  NaN
4      ham  NaN
...    ...  ...
1109  spam  NaN
1110   ham  NaN
1111   ham  NaN
1112   ham  NaN
1113   ham  NaN

[1114 rows x 2 columns]


# Prepare vocabulary - the list of all the words from the dataset

In [145]:
vocabulary = list(set(train_data['SMS'].sum()))

#print(train_data['SMS'])
#vocabulary = list(set(train_data['SMS']))



TypeError: ignored

In [119]:
vocabulary[11:20]

[nan, nan, nan, nan, nan, nan, nan, nan, nan]

In [120]:
len(vocabulary)

4458

# Calculate frequencies of the words for each message

In [142]:
word_counts_per_sms = pd.DataFrame([
    [row[1].count(word) for word in vocabulary]
    for _, row in train_data.iterrows()], columns=vocabulary)

AttributeError: ignored

In [143]:
train_data = pd.concat([train_data.reset_index(), word_counts_per_sms], axis=1).iloc[:,1:]

NameError: ignored

In [ ]:
train_data.head()

# Calculate values for the Bayes formula


In [121]:
alpha = 1

In [122]:
Nvoc = len(train_data.columns) - 3

In [123]:
Pspam = train_data['Label'].value_counts()['spam'] / train_data.shape[0]


In [124]:
Pspam = train_data['Label'].value_counts()['spam'] / train_data.shape[0]


In [130]:
Nspam = train_data.loc[train_data['Label'] == 'spam', 'SMS'].apply(len).sum()

TypeError: ignored

In [131]:
Nham = train_data.loc[train_data['Label'] == 'ham', 'SMS'].apply(len).sum()


TypeError: ignored

In [132]:
def p_w_spam(word):
    if word in train_data.columns:
        return (train_data.loc[train_data['Label'] == 'spam', word].sum() + alpha) / (Nspam + alpha*Nvoc)
    else:
        return 1

In [133]:
def p_w_ham(word):
    if word in train_data.columns:
        return (train_data.loc[train_data['Label'] == 'ham', word].sum() + alpha) / (Nham + alpha*Nvoc)
    else:
        return 1

# Prepare the classificator

In [134]:
def classify(message):
    p_spam_given_message = Pspam
    p_ham_given_message = Pham
    for word in message:
        p_spam_given_message *= p_w_spam(word)
        p_ham_given_message *= p_w_ham(word)
    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_ham_given_message < p_spam_given_message:
        return 'spam'
    else:
        return 'needs human classification'

In [135]:
classify('secret')


NameError: ignored

In [136]:
classify(['secret', 'source', 'of', 'infinite', 'power'])


NameError: ignored

#Use test data

In [137]:
test_data['predicted'] = test_data['SMS'].apply(classify)

NameError: ignored

In [138]:
test_data.head()

,Label,SMS
0,ham,NaN
1,ham,NaN
2,spam,NaN
3,ham,NaN
4,ham,NaN


In [139]:
correct = (test_data['predicted'] == test_data['Label']).sum() / test_data.shape[0] * 100

KeyError: ignored

In [140]:
test_data.loc[test_data['predicted'] != test_data['Label']]


KeyError: ignored

In [141]:
correct


NameError: ignored